# Second round queries

In [1]:
import csv
import os
import re
from collections import defaultdict
from collections import Counter
from datetime import datetime
from datetime import timedelta
from string import punctuation

import matplotlib
import numpy as np
import networkx as nx
import pandas as pd
import requests
import seaborn as sns
from matplotlib import pyplot as plt
from SPARQLWrapper import SPARQLWrapper, JSON

# Autores

In [2]:
%%time

dtypes = {
    'TITULO': str,
    'AUTOR': str,
    'PAIS_EJEMPLAR': str,
    'PAIS_AUTOR': str,
}

autores_df = pd.read_csv('autores.csv', header=0, dtype=dtypes)
autores_df = autores_df.fillna('')

CPU times: user 4.86 s, sys: 218 ms, total: 5.07 s
Wall time: 5.07 s


In [3]:
autores_df.shape

(3054360, 4)

In [4]:
autores_df.head()

,TITULO,PAIS_EJEMPLAR,AUTOR,PAIS_AUTOR
0,Un bicho extraño,Spain,Mon Daporta,DESCONOCIDO
1,Un bicho extraño,Spain,Mon Daporta,DESCONOCIDO
2,Quisiera tener ...,Mexico,Giovanna Zoboli,DESCONOCIDO
3,Johannes Gutenberg,Spain,Lluís Borràs Perelló,DESCONOCIDO
4,La fantástica leyenda de: la princesa y el dragón,Spain,Sonia Alins,DESCONOCIDO


## Query errors

In [5]:
error_autores = set()
error_msg = 'ERROR EN SPARQL PARA'
with open('autores.log') as f:
    for line in f:
        if line.startswith(error_msg):
            autor = line[len(error_msg):].strip()
            error_autores.add(autor)

In [6]:
len(error_autores)

178

## País autor

In [7]:
def get_pais_autor_data_wkd(name, lang):
    template = """
        PREFIX wd: <http://www.wikidata.org/entity/>
        PREFIX wdt: <http://www.wikidata.org/prop/direct/>
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

        SELECT ?authorLabel ?countryLabel
        WHERE {{
            BIND("{0}"@{1} AS ?authorLabel) .
            ?author rdfs:label ?authorLabel .
            ?author wdt:P31 wd:Q5 .
            OPTIONAL {{ ?author wdt:P106 wd:Q36180 . }}
            ?author wdt:P19 ?pob .
            ?pob wdt:P17 ?country .
            ?country rdfs:label ?countryLabel .
            FILTER(lang(?countryLabel)="{1}")
        }}
    """
    query = template.format(name, lang)
    url = 'https://query.wikidata.org/bigdata/namespace/wdq/sparql'
    data = requests.get(url, params={'query': query, 'format': 'json'}).json()
    return data

In [8]:
sparql_dbp = SPARQLWrapper("http://dbpedia.org/sparql")

def get_pais_autor_data_dbp(name, lang):
    template = """
        PREFIX dbo: <http://dbpedia.org/ontology/>
        PREFIX dbp: <http://dbpedia.org/property/>
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

        SELECT ?authorLabel ?countryLabel
        WHERE {{
            BIND("{0}"@{1} AS ?authorLabel) .
            ?author rdfs:label ?authorLabel .
            ?author rdf:type dbo:Person .
            OPTIONAL {{ ?author rdf:type dbo:Writer . }}
            ?author dbp:birthPlace ?pob .
            ?pob rdf:type dbo:Country .
            ?pob rdfs:label ?countryLabel .
            FILTER(lang(?countryLabel)="{1}")
        }}
    """
    query = template.format(name, lang)
    sparql_dbp.setQuery(query)
    sparql_dbp.setReturnFormat(JSON)
    data = sparql_dbp.query().convert()
    return data

In [9]:
def get_pais_autor_data(db, name, lang):
    try:
        if db == 'wdt':
            data = get_pais_autor_data_wkd(name, lang)
        elif db == 'dbp':
            data = get_pais_autor_data_dbp(name, lang)

        results = set(d['countryLabel']['value'] for d in data['results']['bindings'])
        if len(results) == 0:
            result = 'UNKOWN'
        elif len(results) == 1:
            result = results.pop()
        else:
            print('MáS DE UN PAíS PARA', name, ':', results)
            result = results.pop()
    except:
        print('ERROR EN SPARQL PARA', name)
        result = 'UNKOWN'
    return result

In [10]:
def choose(opt_list):
    return Counter(opt_list).most_common(1)[0][0]

In [11]:
# def get_pais_autor(name):
#     r = 'DESCONOCIDO'
#     opt_dict = {}
#     for lang in ['en', 'es']:
#         opt_dict[lang] = {}
#         for db in ['dbp', 'wdt']:
#             opt_dict[lang][db] = get_pais_autor_data(db, name, lang)
#         opts = opt_dict[lang].values()
#         if all(map(lambda x: x != 'UNKOWN', opts)):
#             r = choose(opts)
#             break
#     if r == 'DESCONOCIDO':
#         for lang in ['en', 'es']:
#             opts = opt_dict[lang].values()
#             if any(map(lambda x: x != 'UNKOWN', opts)):
#                 opts = [opt for opt in opts if opt != 'UNKOWN']
#                 r = choose(opts)
#                 break
#     return r

def get_pais_autor(name):
    for lang in ['en', 'es']:
        for db in ['dbp', 'wdt']:
            r = get_pais_autor_data(db, name, lang)
            if r != 'UNKOWN':
                return r
    return 'DESCONOCIDO'

In [12]:
%%time

autores_paises_dict = {}

for i, x in autores_df.drop_duplicates(['AUTOR']).iterrows():
    autores_paises_dict[x['AUTOR']] = x['PAIS_AUTOR']

CPU times: user 18.9 s, sys: 24.2 ms, total: 18.9 s
Wall time: 18.9 s


In [13]:
for x in error_autores:
    autores_paises_dict[x] = get_pais_autor(x)

ERROR EN SPARQL PARA Seminario Interamericano "Participación ciudadana y control
ERROR EN SPARQL PARA Seminario Interamericano "Participación ciudadana y control
ERROR EN SPARQL PARA Seminario Interamericano "Participación ciudadana y control
ERROR EN SPARQL PARA Seminario Interamericano "Participación ciudadana y control
ERROR EN SPARQL PARA Encuentro Institucional de Investigación Formativa "Saberes
ERROR EN SPARQL PARA Encuentro Institucional de Investigación Formativa "Saberes
ERROR EN SPARQL PARA Encuentro Institucional de Investigación Formativa "Saberes
ERROR EN SPARQL PARA Encuentro Institucional de Investigación Formativa "Saberes
ERROR EN SPARQL PARA Jornadas de estudio "La filiación en los inicios de la refle
ERROR EN SPARQL PARA Jornadas de estudio "La filiación en los inicios de la refle
ERROR EN SPARQL PARA Jornadas de estudio "La filiación en los inicios de la refle
ERROR EN SPARQL PARA Jornadas de estudio "La filiación en los inicios de la refle
ERROR EN SPARQL PARA S F

In [14]:
%%time

def process_pais(x):
    return autores_paises_dict[x['AUTOR']]

autores_df['PAIS_AUTOR'] = autores_df.apply(process_pais, axis=1)

CPU times: user 1min 38s, sys: 63.5 ms, total: 1min 38s
Wall time: 1min 38s


# Saving results

In [15]:
autores_df.to_csv('autores2.csv', index=False)